In [ ]:
!pip install openai "numpy<2.0" faiss-cpu scikit-learn

In [ ]:
!pip install sentence-transformers torch

In [ ]:
import numpy as np
import faiss
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time, re, json

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

In [4]:
import configparser
import os
from openai import OpenAI

# --- 설정 파일에서 API 키 불러오기 ---

# ConfigParser 객체 생성
config = configparser.ConfigParser()

# properties 파일 경로 설정
properties_file_path = 'app.properties'

# 파일이 존재하는지 확인 후 읽기
if not os.path.exists(properties_file_path):
    exit(f"오류: 설정 파일 '{properties_file_path}'을(를) 찾을 수 없습니다.")

try:
    config.read(properties_file_path)
    # 'API' 섹션에서 각 키 값 읽어오기
    my_api_key = config.get('API', 'openai.api.key')
    gov_api_key = config.get('API', 'govdata.api.key')
except (configparser.NoSectionError, configparser.NoOptionError) as e:
    exit(f"설정 파일 읽기 오류: {e}")

# OpenAI 클라이언트 초기화
try:
    # 키 값이 비어있는지 확인
    if not my_api_key or my_api_key == 'YOUR_OPENAI_API_KEY_HERE':
        raise ValueError("app.properties 파일에 OpenAI API 키가 설정되지 않았습니다.")
        
    client = OpenAI(api_key=my_api_key)
except (TypeError, ValueError) as e:
    exit(f"OpenAI 클라이언트 초기화 실패: {e}")


# 정부 데이터 API 키 사용 (필요에 따라 활용)
if not gov_api_key or gov_api_key == 'YOUR_GOV_API_KEY_HERE':
    print("경고: app.properties 파일에 정부 데이터 API 키가 설정되지 않았습니다.")


경고: app.properties 파일에 정부 데이터 API 키가 설정되지 않았습니다.


In [ ]:
# GPU(or CPU) 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class PrivacyUtils:
    @staticmethod
    def mask_pii(text: str) -> str:
        text = re.sub(r"(\d{6})[-]\d{7}", r"\1-*******", text)
        text = re.sub(r"(\d{3})[-]\d{2}[-]\d{5}", r"\1-**-*****", text)
        return text

    @staticmethod
    def log_securely(message: str):
        print(PrivacyUtils.mask_pii(message))

In [ ]:
# 벡터DB를 정의한다
# Tf-idf 기반
class VectorDB_tfidf:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.documents = {} # {doc_id: content}
        self.metadata_store = {} # {doc_id: metadata}
        self.doc_vectors = None
        self.doc_ids = []

    def build_index(self):
        if not self.documents:
            self.doc_vectors = None
            self.doc_ids = []
            return

        self.doc_ids = list(self.documents.keys())
        doc_contents = [self.documents[id] for id in self.doc_ids]
        self.doc_vectors = self.vectorizer.fit_transform(doc_contents)

    def search(self, query: str, k: int = 1) -> list[tuple[float, dict]]:
        if self.doc_vectors is None or self.doc_vectors.shape[0] == 0:
            return []
        query_vector = self.vectorizer.transform([query])
        scores = cosine_similarity(query_vector, self.doc_vectors).flatten()
        top_k_indices = scores.argsort()[-k:][::-1]

        return [(scores[i], self.doc_ids[i]) for i in top_k_indices if scores[i] > 0]


In [ ]:
# SentenceTransformer를 이용한 개선된 시맨틱 벡터DB
# 단순 tfIdf 기법은 단어 유사도만 파악하므로 시멘틱으로 단어/문장 의미를 파악
class VectorDB_semantic :
    # 'distiluse-base-multilingual-cased-v1' 모델은 범용 문장 이해 능력이 탁월하나, 일반적인 단어 의미에 집중한다(일반화의 함정)
    # ex) 소상공인 정책자금 대출 : '대출', '정책' 에 높은 가중치, '혁신성장 지원평가 대출' 은 '기술평가' 보다 '대출'에 높은 연관성을 주게 됨
    # def __init__(self, model_name='distiluse-base-multilingual-cased-v1'):
    def __init__(self, model_name='jhgan/ko-sroberta-multitask'):
        # TfidfVectorizer 대신, 의미를 이해하는 언어 모델 로드
        self.model = SentenceTransformer(model_name, device=device)
        self.documents = {}  # {doc_id: content}
        self.metadata_store = {}  # {doc_id: metadata}
        self.doc_vectors = None
        self.doc_ids = []
        # FAISS 인덱스 초기화 (모델의 벡터 차원 수에 맞게)
        self.index = faiss.IndexIDMap(faiss.IndexFlatIP(self.model.get_sentence_embedding_dimension()))

    def build_index(self):
        """저장된 모든 문서를 시맨틱 벡터로 변환하여 FAISS 인덱스에 추가"""

        if not self.documents:
            # 인덱스 초기화
            self.index = faiss.IndexIDMap(faiss.IndexFlatIP(self.model.get_sentence_embedding_dimension()))
            self.doc_ids = []
            return

        self.doc_ids = list(self.documents.keys())
        doc_contents = [self.documents[id] for id in self.doc_ids]

        # model.encode를 사용하여 의미 벡터 생성
        print("  [VectorDB] 문서들을 의미 벡터로 변환 중...")
        self.doc_vectors = self.model.encode(doc_contents, convert_to_tensor=False, normalize_embeddings=True)

        # FAISS 인덱스 재생성
        self.index.reset()
        # FAISS는 numpy array의 ID로 정수만 받으므로, 순차적인 ID를 생성하여 매핑
        self.index.add_with_ids(self.doc_vectors.astype('float32'), np.arange(len(self.doc_ids)))
        print("  [VectorDB] 인덱스 구축 완료.")

    def search(self, query: str, k: int = 1) -> list[tuple[float, dict]]:
        """질문을 의미 벡터로 변환하여 가장 유사한 문서를 검색"""

        if self.index.ntotal == 0:
            return []

        # 질문을 의미 벡터로 변환
        query_vector = self.model.encode([query], convert_to_tensor=False, normalize_embeddings=True)

        # FAISS를 이용한 검색
        scores, indices = self.index.search(query_vector.astype('float32'), k)

        results = []
        for i, score in zip(indices[0], scores[0]):
            if i != -1:
                doc_id = self.doc_ids[i]
                results.append((score, doc_id))
        return results

In [ ]:
# 하이브리드(tfidf + semantic) 검색을 위한 dual-index VectorDB
# 시멘틱 기법은 문장의 의미에만 집중하므로, 정확한 의도 파악이 어려울 수 있다
# 시멘틱 검색으로 문장 단위의 해석 수행하고, tfidf 로 사용자가 필요로 하는 키워드를 탐색
class VectorDB_hybrid:
    def __init__(self, model_name='jhgan/ko-sroberta-multitask'):
        # 1. 의미 기반 검색 엔진
        print(f"  [VectorDB] 시맨틱 검색 모델 '{model_name}' 로드 중...")
        self.semantic_model = SentenceTransformer(model_name, device=device)
        self.faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(self.semantic_model.get_sentence_embedding_dimension()))

        # 2. 키워드 기반 검색 엔진
        self.keyword_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = None

        # 공통 데이터 저장소
        self.documents = {}
        self.metadata_store = {}
        self.doc_ids = []

    def build_index(self):
        if not self.documents: return

        self.doc_ids = list(self.documents.keys())
        doc_contents = [self.documents[id] for id in self.doc_ids]

        # 의미 기반 인덱스 구축
        print("  [VectorDB] 의미 기반 인덱스(FAISS) 구축 중...")
        semantic_vectors = self.semantic_model.encode(doc_contents, convert_to_tensor=False, normalize_embeddings=True)
        self.faiss_index.reset()
        self.faiss_index.add_with_ids(semantic_vectors.astype('float32'), np.arange(len(self.doc_ids)))

        # 키워드 기반 인덱스 구축
        print("  [VectorDB] 키워드 기반 인덱스(TF-IDF) 구축 중...")
        self.tfidf_matrix = self.keyword_vectorizer.fit_transform(doc_contents)
        print("  [VectorDB] 모든 인덱스 구축 완료.")

    def semantic_search(self, query: str, k: int) -> list[tuple[float, str]]:
        """의미가 유사한 문서를 검색"""
        if self.faiss_index.ntotal == 0: return []
        query_vector = self.semantic_model.encode([query], convert_to_tensor=False, normalize_embeddings=True)
        scores, indices = self.faiss_index.search(query_vector.astype('float32'), k)
        return [(scores[0][i], self.doc_ids[idx]) for i, idx in enumerate(indices[0]) if idx != -1]

    def keyword_search(self, query: str, k: int) -> list[tuple[float, str]]:
        """키워드가 일치하는 문서를 검색"""
        if self.tfidf_matrix is None: return []
        query_vector = self.keyword_vectorizer.transform([query])
        scores = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        top_k_indices = scores.argsort()[-k:][::-1]
        return [(scores[i], self.doc_ids[i]) for i in top_k_indices if scores[i] > 0]


In [ ]:
# LLM에 쓰일 RAG 를 정의한다
class RAG_System:
    def __init__(self):
        # 기본은 tf-idf 이용
        self.db = VectorDB_tfidf()

    def set_database(db) :
        """RAG 시스템에 쓰일 데이터베이스를 설정한다"""
        self.db = db

    def add_document(self, doc_id: str, content: str, metadata: dict, build_index: bool = True):
        """외부에서 문서 추가"""
        print(f"  [Knowledge Base] ADD: '{doc_id}' 문서 추가")
        self.db.documents[doc_id] = content
        self.db.metadata_store[doc_id] = metadata
        if build_index: self.db.build_index()

    def delete_document(self, doc_id: str, build_index: bool = True):
        """외부에서 문서 삭제"""
        if doc_id in self.db.documents:
            print(f"  [Knowledge Base] DELETE: '{doc_id}' 문서 삭제")
            del self.db.documents[doc_id]
            del self.db.metadata_store[doc_id]
            if build_index: self.db.build_index()

    def print_documents(self):
        """벡터DB 전체 내용 출력"""
        print("\n" + "="*20 + " RAG Knowledge Base Full Dump " + "="*20)
        if not self.db.documents:
            print("지식 베이스가 비어있습니다.")
        for doc_id, content in self.db.documents.items():
            print(f"\n--- Document ID: {doc_id} ---")
            print(f"  [Content] - {content}")
            print(f"  [Metadata] - {json.dumps(self.db.metadata_store.get(doc_id, {}), ensure_ascii=False)}")
        print("\n" + "="*64)

    # hybrid 검색엔진 이용을 위한 함수 정의
    def hybrid_search(self, query: str, k: int = 5) -> list[str]:
        """ VectorDB 의 두개의 index 검색 결과를 조합하여 최종 순위를 매기는 하이브리드 검색"""
        # 1. 각 엔진으로 K개의 결과 검색
        semantic_results = self.db.semantic_search(query, k=k)
        keyword_results = self.db.keyword_search(query, k=k)

        # 2. RRF(Reciprocal Rank Fusion)를 이용한 점수 재계산
        rrf_scores = {}
        k_rrf = 60  # RRF 알고리즘의 상수로, 보통 60을 사용

        # 의미 검색 결과에 대한 RRF 점수 계산
        for rank, (score, doc_id) in enumerate(semantic_results):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = 0
            rrf_scores[doc_id] += 1 / (k_rrf + rank + 1)

        # 키워드 검색 결과에 대한 RRF 점수 계산
        for rank, (score, doc_id) in enumerate(keyword_results):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = 0
            rrf_scores[doc_id] += 1 / (k_rrf + rank + 1)

        if not rrf_scores:
            return []

        # 3. 최종 점수가 높은 순으로 정렬하여 문서 ID 반환
        sorted_docs = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
        return sorted_docs



In [ ]:
import requests

class AgentToolbox:
    def __init__(self, rag_system):
        self.rag_system = rag_system

    def search_knowledge_base(self, query: str) -> str:
        """사용자 질문과 가장 관련된 정책 정보를 지식 베이스에서 검색합니다."""

        if(isinstance(self.rag_system.db, VectorDB_hybrid)) :
            results = self.rag_system.db.hybrid_search(query, k=1)
        else :
            results = self.rag_system.db.search(query, k=1)

        if not results: return "관련 정보를 찾지 못했습니다."
        score, doc_id = results[0]
        if score < 0.1: return "관련 정보를 찾지 못했습니다. 좀 더 구체적인 키워드로 질문해주세요."

        content = self.rag_system.db.documents[doc_id]
        metadata = self.rag_system.db.metadata_store[doc_id]
        return json.dumps({"content": content, "metadata": metadata}, ensure_ascii=False)

    def fetch_document_from_mcp(self, document_name: str, user_id: str) -> str:
        """MCP를 통해 사용자 동의를 받고 기관에서 서류를 가져오는 것을 시뮬레이션합니다."""
        print(f"  [Tool: MCP] '{document_name}' 전송 요청 (사용자: {user_id})... 사용자 동의 획득...")
        time.sleep(1) # 시뮬레이션 딜레이
        # 데이터 원문 대신, 유효기간 등 메타데이터를 포함한 확인 토큰 반환
        return json.dumps({
            "status": "success", "doc_token": f"TOKEN_{np.random.randint(1000, 9999)}",
            "doc_name": document_name, "issue_date": "2025-07-15",
            "message": "서류 발급 성공"
        })

    def validate_document(self, doc_token: str, issue_date_str: str) -> str:
        """시스템이 서류의 유효성을 자동으로 검증하는 것을 시뮬레이션합니다."""
        print(f"  [Tool: Validator] '{doc_token}' 유효성 검증 시작 (발급일: {issue_date_str})...")
        time.sleep(1)
        # 시나리오: 발급일이 2025-07-01 이후여야 유효하다고 가정
        if issue_date_str >= "2025-07-01":
            return json.dumps({"is_valid": True, "message": "최신 서류로 확인되어 유효합니다."})
        else:
            return json.dumps({"is_valid": False, "message": "서류 유효기간이 만료되었습니다."})

    def submit_application(self, doc_tokens: list, destination: str) -> str:
        """검증된 서류 토큰들을 모아 최종 기관에 제출하는 것을 시뮬레이션합니다."""
        print(f"  [Tool: Submitter] 서류({doc_tokens})를 '{destination}'에 제출합니다...")
        time.sleep(1)
        return json.dumps({"submission_status": "success", "application_id": f"APP_{np.random.randint(1000,9999)}"})

    # [추가] 실제 작동하는 국세청 API 연동 도구(사업자등록번호 상태조회)
    def verify_business_registration(self, user: dict) -> str:
        """ 국세청 API를 호출하여 사업자등록번호의 상태를 확인합니다."""

        print(f"  [Tool: 국세청 API] user '{user}'의 사업자 상태 조회 시작...")

        business_id = user.get('business_id')
        user_id = user.get('user_id')
        if not business_id:
            return json.dumps({"status": "error", "message": "user 객체에 business_id가 없습니다."})

        PrivacyUtils.log_securely(f"  [Internal] Securely retrieved business_id: {business_id} for user_id: {user_id}")

        # 2. 조회된 실제 정보로 외부 API 호출
        api_url = f"https://api.odcloud.kr/api/nts-businessman/v1/status?serviceKey={gov_api_key}"
        payload = {"b_no": [business_id.replace("-", "")]}

        try:
            response = requests.post(api_url, json=payload)
            response.raise_for_status()
            data = response.json()

            if data.get("data") and len(data["data"]) > 0:
                status = data["data"][0]
                tax_type = status.get("tax_type", "정보 없음")
                return json.dumps({
                    "status": "success", "business_id": status.get("b_no"),
                    "taxpayer_status": tax_type, "message": f"조회 성공: {tax_type}"
                }, ensure_ascii=False)
            else:
                return json.dumps({"status": "error", "message": data.get("message", "유효하지 않거나 정보가 없는 사업자번호입니다.")})
        except requests.exceptions.RequestException as e:
            return json.dumps({"status": "error", "message": f"API 호출 중 네트워크 오류 발생: {e}"})

    def synchronize_knowledge_base(self, filepath: str) -> str:
        """크롤링된 최신 데이터 파일과 현재 지식 베이스를 비교하여 동기화합니다."""
        print(f"  [Tool: RAG Sync] '{filepath}' 파일과 동기화를 시작합니다...")

        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                latest_policies = json.load(f)
        except FileNotFoundError:
            return "오류: 동기화할 파일을 찾을 수 없습니다. 크롤러가 먼저 실행되어야 합니다."

        # 4. AI(LLM)를 통해 동기화 계획 수립 (무엇을 추가/삭제/수정할지)
        current_docs_summary = json.dumps(list(self.rag_system.db.documents.keys()))
        latest_policies_summary = json.dumps([p.get("policy_id") for p in latest_policies])

        prompt = f"""
        당신은 RAG 데이터베이스 관리 AI입니다. 현재 데이터베이스와 최신 파일 정보를 비교하여 동기화 계획을 세우세요.
        - 현재 DB 문서 ID: {current_docs_summary}
        - 최신 파일 정책 ID: {latest_policies_summary}

        'add', 'delete', 'update' 해야 할 'policy_id' 목록을 JSON 형식으로 알려주세요.
        """
        response = client.chat.completions.create(
            model="gpt-4o", messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        sync_plan = json.loads(response.choices[0].message.content)

        # 동기화 계획 실행
        added_count, deleted_count = 0, 0
        latest_policies_map = {p["policy_id"]: p for p in latest_policies}

        for doc_id in sync_plan.get("delete", []):
            self.rag_system.delete_document(doc_id, build_index=False)
            deleted_count += 1

        for doc_id in sync_plan.get("add", []) + sync_plan.get("update", []):
            policy = latest_policies_map[doc_id]
            self.rag_system.add_document(
                doc_id=policy["policy_id"],
                content=f"{policy['title']}: {policy['summary']}",
                metadata={"source": "소진공(자동 동기화)", "required_docs": policy['required_docs']},
                build_index=False
            )
            added_count += 1

        self.rag_system.db.build_index()
        result_message = f"동기화 완료: {added_count}개 추가/수정, {deleted_count}개 삭제됨."
        print(f"  [Tool: RAG Sync] {result_message}")

        return result_message

In [ ]:
class AIAgent:
    def __init__(self, user_id: str, rag_system, user_database):
        self.user_id = user_id
        self.USER_DB = user_database
        self.toolbox = AgentToolbox(rag_system)
        self.user = self.USER_DB.get(user_id, {"user_id": user_id})

        self.available_tools = { # 실행할 함수 매핑
            "search_knowledge_base": self.toolbox.search_knowledge_base,
            "fetch_document_from_mcp": self.toolbox.fetch_document_from_mcp,
            "validate_document": self.toolbox.validate_document,
            "submit_application": self.toolbox.submit_application,
            "verify_business_registration": self.toolbox.verify_business_registration,
            "synchronize_knowledge_base": self.toolbox.synchronize_knowledge_base,
        }
        # OpenAI API에 전달할 도구 명세 정의
        self.api_tools = [
            {"type": "function", "function": {"name": "search_knowledge_base", "description": "사용자 질문과 가장 관련된 정책 정보를 지식 베이스에서 검색합니다.", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "사용자의 원본 질문"}}, "required": ["query"]}}},
            {"type": "function", "function": {"name": "fetch_document_from_mcp", "description": "필요한 서류를 MCP를 통해 기관에서 가져옵니다.", "parameters": {"type": "object", "properties": {"document_name": {"type": "string", "description": "가져올 서류의 정확한 이름"}, "user_id": {"type": "string", "description": "요청하는 사용자의 ID"}}, "required": ["document_name", "user_id"]}}},
            {"type": "function", "function": {"name": "validate_document", "description": "가져온 서류가 유효한지(예: 유효기간) 검증합니다.", "parameters": {"type": "object", "properties": {"doc_token": {"type": "string", "description": "검증할 서류의 확인 토큰"}, "issue_date_str": {"type": "string", "description": "서류의 발급일자(YYYY-MM-DD 형식)"}}, "required": ["doc_token", "issue_date_str"]}}},
            {"type": "function", "function": {"name": "submit_application", "description": "모든 검증된 서류를 모아 최종 목적지에 제출합니다.", "parameters": {"type": "object", "properties": {"doc_tokens": {"type": "array", "items": {"type": "string"}}, "destination": {"type": "string", "description": "제출할 기관 이름"}}, "required": ["doc_tokens", "destination"]}}},
            # {"type": "function", "function": {"name": "verify_business_registration", "description": "사용자의 사업자등록 상태가 유효한지를 확인합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "상태를 조회할 사용자의 고유 ID(예 : user_id)"}}, "required": ["user_id"]}}},
            {"type": "function", "function": {
                "name": "verify_business_registration",
                "description": "사용자의 사업자등록 상태가 유효한지 확인합니다. 이 도구는 별도의 파라미터 없이 호출하면, 현재 사용자의 정보로 자동 조회됩니다.",
                "parameters": {"type": "object", "properties": {}} # 파라미터를 비워서 규약 2를 적용
            }},
            {"type": "function", "function": {"name": "synchronize_knowledge_base", "description": "외부 소스로부터 RAG 지식 베이스를 최신 상태로 동기화합니다.", "parameters": {"type": "object", "properties": {"filepath": {"type": "string"}}, "required": ["filepath"]}}},
        ]

    # GateKeeper Filter 함수
    # LLM이 사람을 돕도록 System prompt 가 있어 서비스 외 질문에도 답변을 해버린다
    # 이러한 현상을 해결하기 위해 맨 앞에서 서비스 의도 질문인지를 분류해버림
    def _is_query_in_scope(self, query: str) -> bool:
        print("  [Gatekeeper] 사용자 질문의 의도를 분류합니다...")

        # 의도 분류만을 위한 매우 구체적이고 단순한 프롬프트
        system_prompt = """
            당신은 사용자 질문의 핵심 의도가 '대한민국의 행정 또는 금융 신청 업무'와 관련 있는지 판단하는 분류 전문가입니다.
            사용자의 궁극적인 목표가 대출, 지원금, 계좌 개설, 서류 발급 등과 관련 있다면 'YES'입니다.

            **판단 예시:**
            - 질문: "IT 스타트업을 차릴 건데, 사업자금 대출 알려줘." -> YES
            - 질문: "가게 운영자금이 부족해요." -> YES
            - 질문: "청년도약계좌 만들고 싶어요." -> YES
            - 질문: "오늘 날씨 어때?" -> NO
            - 질문: "낚시하는 법 알려줘" -> NO

            다른 어떤 설명도 하지 말고, 오직 'YES' 또는 'NO'로만 대답하세요.
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o", # 또는 더 저렴한 gpt-3.5-turbo 사용 가능
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query}
                ],
                max_tokens=2, # 답변을 'YES' 또는 'NO'로 제한
                temperature=0.0
            )
            decision = response.choices[0].message.content.strip().upper()
            print(f"  [Gatekeeper] 판단 결과: {decision}")
            return decision == "YES"
        except Exception as e:
            print(f"  [Gatekeeper] 의도 분류 중 오류 발생: {e}")
            return False # 오류 발생 시 보수적으로 접근하여 거절

    # 실제 에이전트 실행
    def run(self, initial_query: str):
        print(f"\n{'#'*10} AI Agent Process Start (Query: '{initial_query}') {'#'*10}")

        # LLM 에 민감정보를 던지지 않기 위해 임의의 id를 내부 Database 에서 조회한다
        contextual_query = f"""
            [사용자 ID]
            {self.user.get('user_id', 'N/A')}

            [사용자 요청 사항]
            {initial_query}
        """

        # GateKeeper 에 의해 질문의 정상 여부(서비스 목적에 맞는지) 확인
        if not self._is_query_in_scope(initial_query):
            refusal_message = "죄송합니다. 저는 대한민국의 행정 및 금융 신청을 돕기 위해 설계된 전문 AI 에이전트입니다. 문의하신 내용에 대해서는 답변을 드리기 어렵습니다. '소상공인 대출'이나 '청년도약계좌' 등 도움이 필요한 신청 업무가 있으시다면 말씀해주세요."
            print("\n[AI-Linker 최종 답변]")
            print(refusal_message)
            print(f"\n{'#'*10} AI Agent Process Finished (Out of Scope) {'#'*10}")
            return # 프로세스 즉시 종료

        # --- 이 아래는 '문지기'를 통과한 경우에만 실행됩니다 ---
        messages = [
            {"role": "system",
             "content": """
                ### 역할 정의 ###
                당신은 'AI-Linker'입니다. 당신의 유일한 임무는 '대한민국의 행정 및 금융 신청 업무'를 자동화하여 사용자를 돕는 것입니다.
                당신에게 전달된 모든 사용자 요청은 이미 관련성 검사를 통과했습니다. 당신은 질문의 의도를 의심할 필요 없이, 오직 아래의 업무 수행 계획에 따라 목표를 완수하는 데만 집중하세요.
                당신은 오직 '사용자 ID'를 통해서만 사용자를 식별하며, 절대 실제 개인정보를 묻거나 다루지 않습니다.
                도구를 사용할 때는, 반드시 [사용자 ID] 컨텍스트로 제공된 값을 그대로 사용해야 합니다.
                절대로 임의의 ID나 예시 값을 만들어서 사용하면 안 됩니다.


                ### **[매우 중요한 업무 수행 계획 (SOP)]** ###
                당신은 반드시 다음의 논리적 순서에 따라 단계별로 계획을 세우고 도구를 사용해야 합니다.

                **0. 지식 동기화:**
               - 가장 먼저, `synchronize_knowledge_base` 도구를 사용해 `latest_policies.json` 파일과 당신의 지식을 동기화하여 최신 상태를 유지합니다.

                **1. 정보 검색 단계:**
                - 가장 먼저, 사용자의 질문 의도를 파악하여 `search_knowledge_base` 도구를 사용해 관련 정책 정보를 검색합니다.
                - 만약 검색 결과가 "관련 정보를 찾지 못했습니다" 라면, 더 이상 다른 도구를 사용하지 말고 사용자에게 이 사실을 알리고 프로세스를 종료합니다.

                **2. 사업자 상태 확인:**
                - 먼저, `사용자 ID`를 `verify_business_registration` 도구에 전달하여 국세청 상태를 확인합니다.
                - 상태가 정상이 아니면 프로세스를 중단합니다.

                **3. 서류 수집 및 검증 단계:**
                - 정보 검색에 성공했다면, 결과에 포함된 **`metadata`의 `required_docs` 리스트**를 확인합니다.
                - 리스트에 있는 **모든 서류에 대해**, `fetch_document_from_mcp` 도구를 **하나씩 순서대로 호출**하여 서류를 가져옵니다.
                - 각 서류를 가져온 직후, 즉시 `validate_document` 도구를 사용하여 해당 서류가 유효한지 검증합니다.

                **4. 최종 제출 단계:**
                - 모든 서류의 수집 및 검증이 성공적으로 완료되었다면, 확보한 모든 `doc_token`들을 모아 `submit_application` 도구를 호출하여 최종 제출을 완료합니다.
                """
            },
            # {"role": "user", "content": initial_query}
            {"role": "user", "content": contextual_query} # user_id 로 되어있는 부분을 이용하도록 유도
        ]

        for i in range(7): # 최대 7단계 실행
            print(f"\n--- Agent Step {i+1} ---")

            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                tools=self.api_tools,
                tool_choice="auto"
            )
            response_message = response.choices[0].message
            messages.append(response_message) # 어시스턴트의 답변을 대화 기록에 추가

            print(f"[response_message] {response_message}")

            # 1. AI가 Tool 사용을 결정했는지 확인
            if response_message.tool_calls:
                print("  [Thought] 도구를 사용해야겠다고 생각했습니다.")
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name

                    function_args = json.loads(tool_call.function.arguments)

                    print(f"  [Action] {function_name}({', '.join(f'{k}={v}' for k, v in function_args.items())})")

                    # 2. 해당 Tool 실행
                    function_to_call = self.available_tools[function_name]

                    # parameter 를 넘기지 않는 경우 해결
                    # 하드 코딩하는 것이 최선의 방안인가?
                    if not function_args :
                        tool_output = function_to_call(user=self.user)
                    else :
                        tool_output = function_to_call(**function_args)

                    print(f"  [Observation] 도구 실행 결과: {tool_output}")

                    # 3. Tool 실행 결과를 대화 기록에 추가
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": tool_output,
                    })
            else:
                # Tool을 사용하지 않고 최종 답변을 한 경우
                print("  [Thought] 모든 작업이 완료되었거나, 더 이상 할 작업이 없다고 판단했습니다.")
                PrivacyUtils.log_securely(response_message.content)
                break

        print(f"\n{'#'*10} AI Agent Process Finished {'#'*10}")


In [ ]:
# RAG system 정의
my_rag_system = RAG_System()

# 개선된 시멘틱DB 이용
my_rag_system.set_database = VectorDB_hybrid()

# RAG에 정책 데이터 추가하기 (소상공인 직접대출)
my_rag_system.add_document(
    doc_id="SBC_LOAN",
    content="소상공인 정책자금 직접대출은 소상공인의 자금 조달을 돕습니다. 필수 서류는 사업자등록증명원, 국세납세증명서입니다.",
    metadata={
        "source": "소상공인시장진흥공단", "destination": "소상공인시장진흥공단", "policy_code": "SBC-DIRECT-2025-Q3",
        "required_docs": ["사업자등록증명원", "국세납세증명서"]
    }
)

# RAG에 정책 데이터 추가하기 (청년도약계좌)
my_rag_system.add_document(
    doc_id="YOUTH_ACCOUNT",
    content="""
        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.
        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.
        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.
        """,
    metadata={
        "source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025",
        "required_docs": ["소득확인증명서", "연령 확인용 신분증"]
    }
)

  [VectorDB] 시맨틱 검색 모델 'jhgan/ko-sroberta-multitask' 로드 중...
  [Knowledge Base] ADD: 'SBC_LOAN' 문서 추가
  [Knowledge Base] ADD: 'YOUTH_ACCOUNT' 문서 추가


In [ ]:
# RAG에 저장된 정책 정보 보기
my_rag_system.print_documents()


==================== RAG Knowledge Base Full Dump ====================

--- Document ID: SBC_LOAN ---
  [Content] - 소상공인 정책자금 직접대출은 소상공인의 자금 조달을 돕습니다. 필수 서류는 사업자등록증명원, 국세납세증명서입니다.
  [Metadata] - {"source": "소상공인시장진흥공단", "destination": "소상공인시장진흥공단", "policy_code": "SBC-DIRECT-2025-Q3", "required_docs": ["사업자등록증명원", "국세납세증명서"]}

--- Document ID: YOUTH_ACCOUNT ---
  [Content] - 
        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.
        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.
        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.
        
  [Metadata] - {"source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025", "required_docs": ["소득확인증명서", "연령 확인용 신분증"]}



In [ ]:
USER_DATABASE = {
        "user_kim": {
            "user_id" : "user_kim",
            "name": "김대표", "business_id": "174-82-00063",
            "resident_id": "950101-1234567"
        }
    }


In [ ]:
    agent = AIAgent(
        user_id="user_kim",
        rag_system=my_rag_system,
        user_database=USER_DATABASE
    )

In [ ]:
# 에이전트에게 질문하기
# LLM 도구에 정의되어 있는 임의의 MCP 도구를 이용해서 신청을 진행하는 시나리오
agent.run("우리 가게 운영자금이 필요한데 소상공인 직접대출 신청 좀 해줘.")


########## AI Agent Process Start (Query: '우리 가게 운영자금이 필요한데 소상공인 직접대출 신청 좀 해줘.') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: YES

--- Agent Step 1 ---
[response_message] ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6mQ62sB5QDfMWhnlpmUXm4jm', function=Function(arguments='{"filepath":"latest_policies.json"}', name='synchronize_knowledge_base'), type='function')])
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] synchronize_knowledge_base(filepath=latest_policies.json)
  [Tool: RAG Sync] 'latest_policies.json' 파일과 동기화를 시작합니다...
  [Observation] 도구 실행 결과: 오류: 동기화할 파일을 찾을 수 없습니다. 크롤러가 먼저 실행되어야 합니다.

--- Agent Step 2 ---
[response_message] ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oBXQhAobKah6d5ABikcyRoMu', function=Function(argum

In [ ]:
# 에이전트에게 질문하기
# LLM 도구에 정의되어 있는 임의의 MCP 도구를 이용해서 신청을 진행하는 시나리오
agent.run("청년도약계좌를 만들고 싶어. 어떻게 해야하지?")


########## AI Agent Process Start (Query: '청년도약계좌를 만들고 싶어. 어떻게 해야하지?') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: YES

--- Agent Step 1 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] search_knowledge_base(query=청년도약계좌 신청 방법)
  [Observation] 도구 실행 결과: {"content": "\n        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.\n        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.\n        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.\n        ", "metadata": {"source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025", "required_docs": ["소득확인증명서", "연령 확인용 신분증"]}}

--- Agent Step 2 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] verify_business_registration()
  [Tool: 국세청 API] user '{'user_id': 'user_kim', 'name': '김대표', 'business_id': '174-82-00063', 'resident_id': '950101-1234567'}'의 사업자 상태 조회 시작...
  [Internal] Securely retrieved business_id: 174-**-***** for user_id: user_kim
  [Observation] 도구 실행 결과: {"status": "success", "business_id

In [ ]:
# RAG에 저장된 정책 정보 다시 보기(크롤링 업데이트)
my_rag_system.print_documents()


==================== RAG Knowledge Base Full Dump ====================

--- Document ID: SBC_LOAN ---
  [Content] - 소상공인 정책자금 직접대출은 소상공인의 자금 조달을 돕습니다. 필수 서류는 사업자등록증명원, 국세납세증명서입니다.
  [Metadata] - {"source": "소상공인시장진흥공단", "destination": "소상공인시장진흥공단", "policy_code": "SBC-DIRECT-2025-Q3", "required_docs": ["사업자등록증명원", "국세납세증명서"]}

--- Document ID: YOUTH_ACCOUNT ---
  [Content] - 
        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.
        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.
        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.
        
  [Metadata] - {"source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025", "required_docs": ["소득확인증명서", "연령 확인용 신분증"]}

--- Document ID: p-101 ---
  [Content] - 소상공인 정책자금 직접대출 공고: 성장 가능성이 높은 소상공인의 자금 조달을 지원합니다.
  [Metadata] - {"source": "소진공(자동 동기화)", "required_docs": ["사업자등록증명원", "국세납세증명서"]}

--- Document ID: p-205 ---
  [Content] - 긴급경영안정자금 (v1753317608.4861379): 매출이 급감한 소상공인을 위한 3분기 긴급 자금입니다.
  [Metadata] - {"

In [ ]:
# 에이전트에게 질문하기
# 혁신성장 지원평가 자금대출
agent.run("내가 이번에 IT 스타트업을 차릴거야. 혁신성장 지원평가 받을 수 있는 대출을 알려줘.")


########## AI Agent Process Start (Query: '내가 이번에 IT 스타트업을 차릴거야. 혁신성장 지원평가 받을 수 있는 대출을 알려줘.') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: YES

--- Agent Step 1 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] search_knowledge_base(query=IT 스타트업 혁신성장 지원평가 대출)
  [Observation] 도구 실행 결과: {"content": "혁신성장 지원자금은 IT 스타트업이나 기술 기반 소상공인을 위한 특별 기술평가 대출입니다. ICT 기술 활용이 필수 요건이며, 기술보증기금의 기술평가서가 필요합니다.", "metadata": {"source": "https://k-startup.go.kr/new_policy.html", "policy_code": "SBC-INNO-2025-Q3", "required_docs": ["사업자등록증명원", "국세납세증명서", "기술평가서"]}}

--- Agent Step 2 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] verify_business_registration()
  [Tool: 국세청 API] user '{'user_id': 'user_kim', 'name': '김대표', 'business_id': '174-82-00063', 'resident_id': '950101-1234567'}'의 사업자 상태 조회 시작...
  [Internal] Securely retrieved business_id: 174-**-***** for user_id: user_kim
  [Observation] 도구 실행 결과: {"status": "success", "business_id": "1748200063", "taxpayer_status": "부가가치세 일반과세자"

In [ ]:
# 에이전트에게 질문하기
# TfIdf 의 한계로 검색어 유사도가 낮으면 제대로 된 답변을 하지 못한다.
# semantic 기반 검색엔진을 벡터DB에 도입하여 정확도를 높일 수 있다
# semantic 기반 검색엔진의 일반화로 잘못된 답변을 도출하는 경우 정밀도가 낮아질 수 있고, tfidf+semantic 혼합한 hybrid 검색을 이용
# Prompt engineering 을 통해 정확도 높은 검색 엔진 개발
agent.run("내가 이번에 IT 스타트업을 차릴거야. 사업자금을 만들고 싶은데 나만의 기발한 기술로 받을 수 있는 대출을 알려줘.")


########## AI Agent Process Start (Query: '내가 이번에 IT 스타트업을 차릴거야. 사업자금을 만들고 싶은데 나만의 기발한 기술로 받을 수 있는 대출을 알려줘.') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: YES

--- Agent Step 1 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] search_knowledge_base(query=기술 기반 IT 스타트업 대출)
  [Observation] 도구 실행 결과: {"content": "혁신성장 지원자금은 IT 스타트업이나 기술 기반 소상공인을 위한 특별 기술평가 대출입니다. ICT 기술 활용이 필수 요건이며, 기술보증기금의 기술평가서가 필요합니다.", "metadata": {"source": "https://k-startup.go.kr/new_policy.html", "policy_code": "SBC-INNO-2025-Q3", "required_docs": ["사업자등록증명원", "국세납세증명서", "기술평가서"]}}

--- Agent Step 2 ---
  [Thought] 도구를 사용해야겠다고 생각했습니다.
  [Action] verify_business_registration()
  [Tool: 국세청 API] user '{'user_id': 'user_kim', 'name': '김대표', 'business_id': '174-82-00063', 'resident_id': '950101-1234567'}'의 사업자 상태 조회 시작...
  [Internal] Securely retrieved business_id: 174-**-***** for user_id: user_kim
  [Observation] 도구 실행 결과: {"status": "success", "business_id": "1748200063", "taxpayer_status": "

In [ ]:
agent.run("낚시를 잘 하는 법을 알려줘.")


########## AI Agent Process Start (Query: '낚시를 잘 하는 법을 알려줘.') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: NO

[AI-Linker 최종 답변]
죄송합니다. 저는 대한민국의 행정 및 금융 신청을 돕기 위해 설계된 전문 AI 에이전트입니다. 문의하신 내용에 대해서는 답변을 드리기 어렵습니다. '소상공인 대출'이나 '청년도약계좌' 등 도움이 필요한 신청 업무가 있으시다면 말씀해주세요.

########## AI Agent Process Finished (Out of Scope) ##########


In [ ]:
agent.run("2025년 7월 21일 현재 파리로 가는 가장 싼 비행기를 알려줘.")


########## AI Agent Process Start (Query: '2025년 7월 21일 현재 파리로 가는 가장 싼 비행기를 알려줘.') ##########
  [Gatekeeper] 사용자 질문의 의도를 분류합니다...
  [Gatekeeper] 판단 결과: NO

[AI-Linker 최종 답변]
죄송합니다. 저는 대한민국의 행정 및 금융 신청을 돕기 위해 설계된 전문 AI 에이전트입니다. 문의하신 내용에 대해서는 답변을 드리기 어렵습니다. '소상공인 대출'이나 '청년도약계좌' 등 도움이 필요한 신청 업무가 있으시다면 말씀해주세요.

########## AI Agent Process Finished (Out of Scope) ##########
